In [25]:
sentences = [
    "오늘 날씨가 좋아서 나들이 가고 싶다.",
    "이 영화는 정말 재미있었어요.",
    "맛있는 음식을 먹으러 갈까요?",
    "운동을 하면 건강에 좋아지는 것 같아요.",
    "공부하기 싫어서 미루고 있어요.",
    "여행 계획을 세우고 있는데 어디로 갈까요?",
    "좋은 책을 읽으면 마음이 편안해져요.",
    "오늘은 친구들과 만나서 재미있게 놀았어요.",
    "새로운 언어를 배우는 것은 어려워도 흥미로워요.",
    "주말에 가족들과 함께 시간을 보내기로 했습니다."
]

In [26]:
from mecab import MeCab
mecab = MeCab()

### 자연어 전처리

In [27]:
# 불용어 리스트 생성 (예시)
stopwords = ['가', '고', '을', '를', '이', '는']
from konlpy.tag import Okt

# Okt 형태소 분석기 인스턴스 생성
okt = Okt()

# 불용어 리스트 생성 (예시)
stopwords = ['가', '고', '을', '를', '이', '는']

# 토크나이징 함수 정의
def tokenizer(raw, pos=["Noun","Alpha","Verb","Number"], stopword=stopwords):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅏ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword

    ]

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer #사전이 중심
vectorizer = TfidfVectorizer(tokenizer=tokenizer,use_idf=True) 
#vectorizer.fit(morphs_list)
features = vectorizer.fit_transform(sentences)
features.toarray()

C:\Users\01-10\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array([[0.46015789, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.46015789, 0.46015789, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.46015789, 0.        , 0.        ,
        0.        , 0.        , 0.39117625, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.70710678, 0.        , 0.        , 0.        ,
        0.        , 0.70710678, 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.51519219, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
      

### LDA 적용

In [29]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model = LatentDirichletAllocation(n_components=3, random_state=111) #top 3개

In [30]:
lda_model.fit(features) # 모델 교육 끝

LatentDirichletAllocation(n_components=3, random_state=111)

In [31]:
# lda_model.components_
# len(lda_model.components_[0])
topics_list = lda_model.components_[0] #첫번째 토픽

### 토픽별 단어표시

In [52]:
dictionary_list = vectorizer.get_feature_names_out()
# len(vectorizer.get_feature_names_out())
dictionary_list


array(['가다', '가족', '갈다', '건강', '계획', '공부', '나들이', '날씨', '놀다', '마음', '만나다',
       '먹다', '미루다', '배우다', '보내다', '세우다', '시간', '싶다', '어디', '언어', '여행',
       '영화', '오늘', '운동', '음식', '읽다', '정말', '좋아지다', '주말', '친구', '하다'],
      dtype=object)

In [33]:
import pandas as pd
df_datas = [topics_list, dictionary_list]
df_topics = pd.DataFrame(data=df_datas)
df_topics = df_topics.T
df_topics[:2]

,0,1
0,0.33441,가다
1,0.799267,가족


In [34]:
df_topics.columns

RangeIndex(start=0, stop=2, step=1)

In [35]:
df_topics.sort_values(0, ascending=False)

,0,1
30,1.539029,하다
19,1.036222,언어
13,1.036222,배우다
12,0.955735,미루다
5,0.955735,공부
3,0.860837,건강
27,0.860837,좋아지다
23,0.860837,운동
1,0.799267,가족
28,0.799267,주말


In [36]:
topics_list = lda_model.components_[1] #첫번째 토픽

In [37]:
import pandas as pd
df_datas = [topics_list, dictionary_list]
df_topics = pd.DataFrame(data=df_datas)
df_topics = df_topics.T
df_topics[:2]

,0,1
0,0.791191,가다
1,0.334677,가족


In [38]:
df_topics.sort_values(0, ascending=False)

,0,1
22,1.162787,오늘
21,1.036857,영화
26,1.036857,정말
8,0.849069,놀다
10,0.849069,만나다
29,0.849069,친구
6,0.791191,나들이
7,0.791191,날씨
17,0.791191,싶다
0,0.791191,가다


### 댓글과 주요 토픽 연결

In [39]:
## 상위 단어 추출
topics_list = list()
for topic in lda_model.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics = df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
    # print(df_topic[:3])
    topics_text = ' '.join(df_topics[1].values[:4]) # get values from Serise
    print(topics_text)
    topics_list.append(topics_text)

하다 언어 배우다 미루다
오늘 영화 정말 놀다
갈다 읽다 마음 먹다


In [40]:
topics_list_add = [topics_list, ['Topic0', 'Topic1', 'Topic2']]
df_topic_keyword = pd.DataFrame(topics_list_add)
df_topic_keyword.T

,0,1
0,하다 언어 배우다 미루다,Topic0
1,오늘 영화 정말 놀다,Topic1
2,갈다 읽다 마음 먹다,Topic2


In [41]:
' '.join(df_topics[1].values[:4])

'갈다 읽다 마음 먹다'

In [42]:
topics_output = lda_model.transform(features)
topics_output

array([[0.10462299, 0.7905512 , 0.10482582],
       [0.13946382, 0.72088285, 0.13965333],
       [0.12347941, 0.12362429, 0.7528963 ],
       [0.77339137, 0.11328861, 0.11332002],
       [0.75164634, 0.12416304, 0.12419063],
       [0.10458222, 0.10474923, 0.79066855],
       [0.13943843, 0.13959605, 0.72096551],
       [0.11268585, 0.77443415, 0.11288   ],
       [0.72035161, 0.13980797, 0.13984042],
       [0.78957076, 0.10519796, 0.10523128]])

In [43]:
lda_model.components_
len(lda_model.components_[0])
# topics_list = lda_model.components_[0] #첫번째 토픽

31

### 각 댓글별 topic 분류

In [44]:
lda_model.n_components

3

In [45]:
df_topics_score = pd.DataFrame(topics_output)

In [46]:
# 각 댓글마다 토픽 분류 지정
import numpy as np
dominant_in_topic = np.argmax(topics_output, axis = 1)

In [47]:
df_topics_score['dominant_topic'] = dominant_in_topic
df_topics_score[:3]

,0,1,2,dominant_topic
0,0.104623,0.790551,0.104826,1
1,0.139464,0.720883,0.139653,1
2,0.123479,0.123624,0.752896,2


In [50]:
df_topics_score['sentesces'] = sentences
df_topics_score

,0,1,2,dominant_topic,sentesces
0,0.104623,0.790551,0.104826,1,오늘 날씨가 좋아서 나들이 가고 싶다.
1,0.139464,0.720883,0.139653,1,이 영화는 정말 재미있었어요.
2,0.123479,0.123624,0.752896,2,맛있는 음식을 먹으러 갈까요?
3,0.773391,0.113289,0.113320,0,운동을 하면 건강에 좋아지는 것 같아요.
4,0.751646,0.124163,0.124191,0,공부하기 싫어서 미루고 있어요.
5,0.104582,0.104749,0.790669,2,여행 계획을 세우고 있는데 어디로 갈까요?
6,0.139438,0.139596,0.720966,2,좋은 책을 읽으면 마음이 편안해져요.
7,0.112686,0.774434,0.112880,1,오늘은 친구들과 만나서 재미있게 놀았어요.
8,0.720352,0.139808,0.139840,0,새로운 언어를 배우는 것은 어려워도 흥미로워요.
9,0.789571,0.105198,0.105231,0,주말에 가족들과 함께 시간을 보내기로 했습니다.
